In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import v_measure_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster, cophenet, ward
from scipy.spatial.distance import pdist
from mpl_toolkits.mplot3d import Axes3D

### irisデータセットの読み込み
中身は4つの特徴ベクトルからなる3種類50個ずつのデータ

In [ ]:
iris = datasets.load_iris()
print(iris.data)
print(iris.target)

試しに、1つめと2つめを特徴ベクトルとして2次元にマッピング

In [ ]:
for i in range(0,len(iris.data)):
    if iris.target[i] == 0:
        plt.plot(iris.data[i][0], iris.data[i][1], "ro")
    elif iris.target[i] == 1:
        plt.plot(iris.data[i][0], iris.data[i][1], "go")
    elif iris.target[i] == 2:
        plt.plot(iris.data[i][0], iris.data[i][1], "bo")
plt.show()

PCA(主成分分析)で3次元にしたデータを可視化。
3次元にした時の累積寄与率も表示

In [ ]:
pca = PCA(n_components=3)
pca.fit(iris.data)
data_reduced = pca.transform(iris.data)
print("explained : %f" % np.cumsum(pca.explained_variance_ratio_)[::-1][0])

In [ ]:
print(iris.data[0:3])
print(data_reduced[0:3])

In [ ]:
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=-150, azim=110)
for i in range(0,len(iris.data)):
    if iris.target[i] == 0:
        ax.scatter(data_reduced[i, 0], data_reduced[i, 1], data_reduced[i, 2], color="r")
    elif iris.target[i] == 1:
        ax.scatter(data_reduced[i, 0], data_reduced[i, 1], data_reduced[i, 2], color="g")
    elif iris.target[i] == 2:
        ax.scatter(data_reduced[i, 0], data_reduced[i, 1], data_reduced[i, 2], color="b")
ax.set_title("First three PCA directions")
ax.set_xlabel("1st eigenvector")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("3rd eigenvector")
ax.w_zaxis.set_ticklabels([])

k-means法でクラスタリング
クラスタ数は3で行ってみる

In [ ]:
clf = KMeans(n_clusters=3,n_jobs=-1,max_iter=1000,n_init=100)
clf.fit(data_reduced)

In [ ]:
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=-150, azim=110)
for i in range(0,len(iris.data)):
    if clf.labels_[i] == 0:
        ax.scatter(data_reduced[i, 0], data_reduced[i, 1], data_reduced[i, 2], color="r")
    elif clf.labels_[i] == 1:
        ax.scatter(data_reduced[i, 0], data_reduced[i, 1], data_reduced[i, 2], color="g")
    elif clf.labels_[i] == 2:
        ax.scatter(data_reduced[i, 0], data_reduced[i, 1], data_reduced[i, 2], color="b")

ax.set_title("K-means result")
ax.set_xlabel("1st eigenvector")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("3rd eigenvector")
ax.w_zaxis.set_ticklabels([])

クラスタの内的妥当性尺度である
* クラスタ内距離二乗和
* pseudoF

の実装

In [ ]:
def make_cluster(data: np.ndarray, label: np.ndarray):
    """
    クラスタリングのラベルをもとにクラスタのリストを作る
    :param data: クラスタリングされたデータ
    :param label: クラスタリング結果の配列
    :return: クラスタごとのデータ集合、クラスタごとの重心
    """

    def sort_smart(lists: list):
        def smart(lists: list):
            """A1 A10 A2みたいなものをスマートに並び替える"""
            convert = lambda text: int(text) if text.isdigit() else text
            alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
            lists.sort(key=alphanum_key)
            return lists
        try:
            return smart(lists)
        except:
            return sorted(lists)

    keys = sort_smart(set(label))
    index = [np.where(label == key)[0] for key in keys]

    cluster = [data[index[i]] for i in range(0, len(index))]
    c_mean = np.array([np.mean(c, axis=0) for c in cluster])

    return cluster, c_mean


def cluster_sum_of_square(data: np.ndarray, label: np.ndarray):
    """
    クラスタ内距離二乗和を求める関数
    :param data: クラスタリングされたデータ
    :param label: クラスタリング結果の配列
    :return: 距離
    """
    cluster, c_mean = make_cluster(data, label)
    dis = [[np.linalg.norm(cluster[i][j] - c_mean[i]) ** 2 for j in range(0, len(cluster[i]))]
           for i in range(0, len(c_mean))]
    return np.sum([np.sum(d) for d in dis])


def pseudof(data: np.ndarray, label: np.ndarray):
    """
    pseudofはクラスタリングの内的妥当性尺度
    :param data: クラスタリングされたデータ
    :param label: クラスタリング結果の配列
    :return:高いほど妥当性が高い
    """
    cluster, c_mean = make_cluster(data, label)
    k = len(c_mean)
    n = len(data)
    d_all = cluster_sum_of_square(data, np.zeros(len(data)))
    d_cluster = cluster_sum_of_square(data, label)
    return ((d_all - d_cluster) / (k - 1)) / (d_cluster / (n - k))


クラスタの重心があっているか確認

In [ ]:
cluster, c_mean = make_cluster(data_reduced,clf.labels_)
print(c_mean)
print(clf.cluster_centers_)

In [ ]:
cluster_sum_of_square(data_reduced,clf.labels_)

In [ ]:
pseudof(data_reduced,clf.labels_)

階層的クラスタリングでも確認
pdistを自作距離から改変することでいい結果が得られそう

In [ ]:
from clustering import HAC

In [ ]:
ac = HAC(iris.data,iris.target)

In [ ]:
dis = [jsd(iris.data[i],iris.data[j]) for i in range(0,len(iris.data) - 1)
       for j in range(i + 1,len(iris.data))]

In [ ]:
ac.set_distance_array(dis,"average")

In [ ]:
ac.draw_dendrogram(p=4, truncate_mode='lastp')
ac.clustering(4)

In [ ]:
cluster_sum_of_square(data_reduced,ac.pred)

In [ ]:
pseudof(data_reduced,ac.pred)

In [ ]:
v_measure_score(iris.target, ac.pred)

In [ ]:
pf = []
cs = []
vm = []
for i in range(1,31):
    ac.clustering(i)
    cs.append(cluster_sum_of_square(data_reduced,ac.pred))
    pf.append(pseudof(data_reduced,ac.pred))
    vm.append(v_measure_score(iris.target, ac.pred))

In [ ]:
plt.plot(pf)
plt.grid()

In [ ]:
plt.plot(cs)
plt.grid()

In [ ]:
plt.plot(vm)
plt.grid()

pseudofとv-measureにおいていずれもクラスタ数3が適切であったことがわかる

In [ ]:
ac.set_tree_structure()

In [ ]:
ac.tree

In [ ]:
ac.c_index

In [ ]:
ac.get_tree_structure(140)

In [ ]:
iris.data[[1,2,3]]

In [ ]:
def pseudot(cls1: np.ndarray, cls2: np.ndarray):
    sum_cls1 = cluster_sum_of_square(cls1,np.zeros(len(cls1)))
    sum_cls2 = cluster_sum_of_square(cls2,np.zeros(len(cls2)))
    n_cls1 = len(cls1)
    n_cls2 = len(cls2)
    
    d_cluster = cluster_sum_of_square(np.vstack([cls1,cls2]),np.zeros(len(np.vstack([cls1,cls2]))))
    
    return d_cluster / ((sum_cls1 + sum_cls2) / (n_cls1 + n_cls2 - 2))

In [ ]:
pt = []
for i in range(0,len(ac.tree)):
    index = ac.get_tree_structure(i)
    cls1 = ac.train[index[0]]
    cls2 = ac.train[index[1]]
    
    pt.append(pseudot(cls1,cls2))
    
plt.plot(list(reversed((range(0,len(pt))))),pt)
plt.xlim(0,10)


### 以下備忘録

In [ ]:
import numpy as np
from scipy import stats


def kl(w1,w2):
    return stats.entropy(w1, w2, 2)

def js(w1, w2):
    r = (w1 + w2) / 2
    return 0.5 * (stats.entropy(w1, r, 2) + stats.entropy(w2, r, 2))

eat = np.array([0.9, 0.1])
devour = np.array([0.8, 0.2])
drink = np.array([0.1, 0.9])

print("kl divergence")
print(kl(devour, eat))
print(kl(devour, drink))

print("js divergence")
print(js(devour, eat))
print(js(devour, drink))

In [ ]:
import numpy as np
import scipy.spatial.distance as distance
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
from random import random
from pandas import DataFrame, Series
 
#10行10列のランダム行列生成
n = 10
data = [[random() for i in range(n)] for i in range(n)]
 
#距離行列生成
dMatrix = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        dMatrix[i, j] = distance.chebyshev(data[i], data[j]) #チェビシェフ距離をとる
 
#距離ベクトル生成
dArray = distance.squareform(dMatrix)
 
#クラスタリング
result = linkage(dArray, method = 'average')
 
#図示
dendrogram(result)
dArray

In [ ]:
n = 100
dim = 10
data = [[random() for i in range(dim)] for i in range(n)]
result1 = linkage(data, metric = 'chebyshev', method = 'average')
dArray1 = distance.pdist(data, metric = 'chebyshev')
result2 = linkage(dArray1, method = 'average')
 
assert (result1 == result2).all()#同じ結果
dArray2 = []
for i in range(n - 1):
    for j in range(i + 1, n):
        dArray2.append(distance.chebyshev(data[i], data[j]));
assert (dArray1 == dArray2).all()#同じもの
dMatrix1 = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        dMatrix1[i, j] = distance.chebyshev(data[i], data[j])
dArray3 = distance.squareform(dMatrix1)
assert (dArray1 == dArray3).all()#同じもの
dMatrix2 = distance.squareform(dArray1)
assert (dMatrix1 == dMatrix2).all()#同じもの

In [ ]:
dArray1

In [ ]:
[distance.chebyshev(data[i],data[j]) for i in range(0,n - 1) for j in range(i + 1,n)]

In [ ]:
def kl(p, q):
    return np.sum(np.where(p != 0, p * np.log2(p / q), 0))

def js(p, q):
    m = (p + q) / 2
    return (kl(p, m) + kl(q, m)) / 2.0

def kld(vec1: list, vec2: list):
    """Calculates Kullback–Leibler divergence"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.sum(vec1 * np.log(vec1 / vec2), axis=(vec1.ndim - 1))

def jsd(vec1: list, vec2: list):
    """Calculates Jensen-Shannon Divergence"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    m = 0.5 * (vec1 + vec2)
    return (0.5 * kld(vec1, m) + 0.5 * kld(vec2, m))

In [ ]:
a = np.array([1,2,3,4,5])
b = np.array([1,2,3,2,1])
print(kl(a,b))
print(kld(a,b))
print(js(a,b))
print(jsd(a,b))

In [ ]:
a*np.log(a/b)

In [ ]:
np.where(a != 0,a * np.log(a/b),0)

In [ ]:
glaph = [0.95,0.94,0.94,0.95,0.95,0.94,0.95,0.95,0.95,0.94,0.94,0.95]
x = []
plt.ylabel("F-measure")
plt.xlabel("Frequency")
plt.plot(glaph,"o-")
plt.ylim(0,1)

In [ ]:
glaph = [0.76,0.88,0.82,0.82,0.79,0.75,0.75,0.77,0.63,0.61,0.51,0.46]
x = []
plt.ylabel("F-measure")
plt.xlabel("Frequency")
plt.plot(glaph,"o-")
plt.ylim(0,1)

In [ ]:
from multiprocessing import Pool,Process,Queue

In [ ]:
pool = Pool()

In [ ]:
def square(x,queue):
    queue.put(x**2)

In [ ]:
def add(x):
    return x*x

In [ ]:
pool.map(square,list(range(0,100000)))

In [ ]:
q = Queue()

In [ ]:
ps = [Process(target=square,args=(i,q)) for i in range(0,10)]

In [ ]:
[p.start() for p in ps]

In [ ]:
a = []
[a.append(q.get()) for p in ps]

In [ ]:
a=[Process(target=add,args=(i,)) for i in range(0,10)]